In [2]:
import matplotlib.pyplot as plt
import numpy as np

# Introduction
This document discusses sampling from Gaussian process (GP) posterior distributions. Since GPs can be considered priors on functions, we can conceptualize this problem as that of sampling functions from some distribution. In practice though, we of course only sample the functions at a finite number of points. Nonetheless, we are often interested in function values at many points, and the function values are typically quite correlated. These features make this sampling task a difficult one.

# Setup and Notation

## GP Regression
Consider the standard GP regression model 

$$y = f(x) + \epsilon$$

where $\epsilon \sim N(0, \sigma^2)$ is independent Gaussian noise. We place a GP prior on the latent function $f$:

$$ f \sim \mathcal{GP}(0, k)$$

where I have assumed the mean function is identically zero for simplicity. Suppose we have observations $(x^\prime_1, y_1), \dots, (x^\prime_N, y_N)$, collected in a $N \times d$ matrix of input locations $\mathbf{X}^\prime$ and an $N \times 1$ response vector $\mathbf{y}$. Also let $\mathbf{f}^\prime$ be the $N \times 1$ vector of (unobserved) latent function values at the points $\mathbf{X}^\prime$. Suppose we also have a set of input locations $x_1, \dots, x_M$ at which we would like to predict, similarly collected in a matrix $\mathbf{X}$. Let $\mathbf{f}$ denote the $M \times 1$ vector of latent function values at these inputs. In general, these locations might include the training locations or not. In this setting, two quantities are typically of primary interest: the marginal likelihood and the predictive (posterior) distribution over function values. The former is given by 
$$ p(\mathbf{y}|\mathbf{X}^\prime) = \int p(\mathbf{y}, \mathbf{f}^\prime|\mathbf{X}^\prime) d\mathbf{f}^\prime = \int p(\mathbf{y}|\mathbf{f}^\prime, \mathbf{X}^\prime)p(\mathbf{f}^\prime|\mathbf{X}^\prime) d\mathbf{f}^\prime$$
The likelihood is called *marginal*, as its computation requires marginalizing out the unobserved latent function values $\mathbf{f}^\prime$ at the training input points. In this setting, this integral is analytically tractable since both terms are Gaussian

\begin{align*}
p(\mathbf{y}|\mathbf{f}^\prime, \mathbf{X}^\prime) &= \mathcal{N}_N(\mathbf{y}|\mathbf{f}^\prime, \sigma^2 \mathbf{I}_N) \\
p(\mathbf{f}^\prime|\mathbf{X}^\prime) &= \mathcal{N}_N(\mathbf{f}^\prime|\mathbf{0}, \mathbf{K}^\prime)
\end{align*}

where $\mathbf{K}^\prime$ is the $N \times N$ matrix resulting from evaluating the kernel at the training inputs. This integral is only analytically tractable due to the simple choice of likelihood and the linear connection between $y$ and $f$. We are interested in more general settings where we lose this analytical tractability. 

The second primary quantity of interest is the posterior over the latent function values $\mathbf{f}$. 
$$p(\mathbf{f}|\mathbf{y}) = \int p(\mathbf{f}|\mathbf{f}^\prime, \mathbf{y})p(\mathbf{f}^\prime|\mathbf{y})d\mathbf{f}^\prime = \int p(\mathbf{f}|\mathbf{f}^\prime)p(\mathbf{f}^\prime|\mathbf{y})d\mathbf{f}^\prime$$
Once again, this requires marginalizing out the latent function values $\mathbf{f}^\prime$ at the training inputs, and once again this is analytically tractable in this simple regression setting. Note that technically I should be conditioning on $\mathbf{X}^\prime$ and $\mathbf{X}$ as well, but I suppress this to lighten notation. This document focuses on sampling from distributions like $p(\mathbf{f}|\mathbf{y})$, but in the more general case where the resulting integral cannot be computed in closed-form. An example of such a problem stems from GP classification.  

## GP Classification
TODO

# Gibbs Sampling
We start simply, considering a Gibbs sampling approach for $p(\mathbf{f}|\mathbf{y})$. In fact, let's start very simply and consider $p(\mathbf{f}|\mathbf{y})$ in the simple regression setting so that we have a known closed-form solution as a basis for comparison. To begin, we must find the Gibbs conditionals $p(f_i|\mathbf{f}_{-i},y)$ where $f_i$ is the $i^{\text{th}}$ entry of $\mathbf{f}$ and $\mathbf{f}_{-i}$ is the vector of length $M - 1$ excluding the $i^{\text{th}}$ entry of $\mathbf{f}$. Unsurprisingly, in the basic regression setting these conditionals are available in closed-form. This can be seen by observing that the vector 

$$ \begin{pmatrix} f_i \\ \mathbf{f}_{-i} \\ \mathbf{y} \end{pmatrix} = \begin{pmatrix} \mathbf{f} \\ \mathbf{y} \end{pmatrix}$$
is joint Gaussian distributed. The mean of this distribution is clearly $\mathbf{0}$. To find the covariance, consider 

$$\text{Cov}(\mathbf{y}, \mathbf{f}) = \text{Cov}(\mathbf{f}^\prime + \mathbf{\epsilon}, \mathbf{f}) = \text{Cov}(\mathbf{f}^\prime, \mathbf{f}) = k(\mathbf{X}^\prime, \mathbf{X})$$
which is a matrix of dimension $N \times M$. The joint Gaussian is thus given by 

$$
\begin{pmatrix} \mathbf{f} \\ \mathbf{y} \end{pmatrix} \sim \mathcal{N}_{MN}\left(\mathbf{0}, 
\begin{pmatrix} \mathbf{K} & k(\mathbf{X}, \mathbf{X}^\prime) \\ \end{pmatrix}\right)
$$